# Predicting Long-Run Future Stock Returns with the Cyclically Adjusted Price-Earnings Ratio (CAPE)

In 1998, Robert Shiller and John Campbell published the pathbreaking article “Valuation Ratios and the Long-Run Stock Market Outlook.” A follow-up to some of their earlier work on stock market predictability, it established that long-term stock market returns were not random walks but, rather, could be forecast by a valuation measure called the “cyclically adjusted price–earnings ratio,” or CAPE ratio. Shiller and Campbell calculated the CAPE ratio by dividing a long-term broad-based index of stock market prices and earnings from 1871 by the average of the last 10 years of earnings per share, with earnings and stock prices measured in real terms. **They regressed 10-year real stock returns against the CAPE ratio and found that the CAPE ratio is a significant variable that can predict long-run stock returns.** The predictability of real stock returns implies that long-term equity returns are mean reverting. In other words, if the CAPE ratio is above (below) its long-run average, the model predicts below average (above-average) real stock returns for the next 10 years. 

*Jeremy J. Siegel (2016) The Shiller CAPE Ratio: A New Look, Financial Analysts Journal, 72:3, 41-50, DOI: 10.2469/faj.v72.n3.1*

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from tqdm import tqdm

In [2]:
classifications = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/classification_data.csv', index_col=0).iloc[:, :-2]
grinold_kroner = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/grinold_kroner_returns.csv', index_col=0)
current_fwd_return_5y_forecast = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/current_fwd_return_5y_forecast.csv', index_col=0)
benchmark_prices = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/benchmark_prices.csv', index_col=0)
benchmark_lt_pe = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/benchmark_lt_pe.csv', index_col=0)
# vif_equity_etfs = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/vif_equity_etf.csv', index_col=0)

classifications.index.name = 'BENCHMARK_TICKER'
grinold_kroner.index.name = 'BENCHMARK_TICKER'
current_fwd_return_5y_forecast.index.name = 'BENCHMARK_TICKER'

In [3]:
results = pd.read_csv('https://raw.githubusercontent.com/nathanramoscfa/cape/main/data/equity_etf_posterior_returns.csv')
results.columns = ['BENCHMARK_TICKER', 'ETF_TICKER', 'CORRELATION', 'P_VALUE', 'BENCHMARK_NAME', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW']
results = results[['ETF_TICKER', 'CORRELATION', 'P_VALUE', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW', 'BENCHMARK_NAME', 'BENCHMARK_TICKER']]
results.ETF_TICKER = results.ETF_TICKER.str.replace(' US Equity', '')
results['ETF_NAME'] = classifications.loc[results.ETF_TICKER.values].NAME.values
results['CLASSIFICATION'] = classifications.loc[results.ETF_TICKER.values].CLASSIFICATION.values
results = results.set_index('ETF_TICKER')
results = pd.merge(results, grinold_kroner, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results = pd.merge(results, current_fwd_return_5y_forecast.FWD_RETURN_5Y_FORECAST, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results = results[results['CORRELATION']>=0.99].sort_values(by='POSTERIOR_RETURN', ascending=False)
results

,CORRELATION,P_VALUE,PRIOR_RETURN,POSTERIOR_RETURN,VIEW,BENCHMARK_NAME,BENCHMARK_TICKER,ETF_NAME,CLASSIFICATION,LONG_TERM_EARNINGS_YIELD,NOMINAL_EARNINGS_GROWTH,REPRICING_RETURN,GRINOLD_KRONER_RETURN,FWD_RETURN_5Y_FORECAST
ETF_TICKER,,,,,,,,,,,,,,
PSCD,0.9993,0,0.184952,0.121506,0.17615,S&P 600 Consumer Discretionary Sector GICS Lev...,S6COND Index,Invesco S&P SmallCap Consumer Discretionary ETF,U.S. Small-cap Value ETP,0.0781,0.0375,0.0178,0.1334,0.2189
SLYV,0.9924,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,SPDR S&P 600 Small CapValue ETF,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593
VIOV,0.9924,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,Vanguard S&P Small-Cap 600 Value ETF,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593
IJS,0.9933,0,0.164868,0.096484,0.13455,S&P Small Cap 600 Value Index,SMLV Index,iShares S&P Small-Cap 600 Value ETF,U.S. Small-cap Value ETP,0.0683,0.0375,0.0040,0.1098,0.1593
PSCI,0.9966,0,0.171587,0.095447,0.10815,S&P 600 Industrials Sector GICS Level 1 Index,S6INDU Index,Invesco S&P SmallCap Industrials ETF,U.S. Small-cap Value ETP,0.0502,0.0375,-0.0031,0.0846,0.1317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IXP,0.9957,0,0.128206,0.022326,0.00195,MSCI World Communication Services Sector Index,MXWO0TC Index,iShares Global Comm Services ETF,Global Large-cap Growth ETP,0.0438,0.0375,-0.0205,0.0608,-0.0569
VOX,0.9966,0,0.128143,0.021969,0.00230,MSCI World ex AUS Communication Services Index,MXWOOTC Index,Vanguard Communication Services ETF,U.S. Large-cap Blend ETP,0.0438,0.0375,-0.0206,0.0607,-0.0561
FCOM,0.9968,0,0.128143,0.021969,0.00230,MSCI World ex AUS Communication Services Index,MXWOOTC Index,Fidelity MSCI Communication Services Index ETF,U.S. Broad Market Blend ETP,0.0438,0.0375,-0.0206,0.0607,-0.0561


In [4]:
df = results[['ETF_NAME', 'BENCHMARK_NAME', 'BENCHMARK_TICKER', 'FWD_RETURN_5Y_FORECAST']]
# df = df[df['BENCHMARK_TICKER'].isin(list(vif_equity_etfs.index))].sort_values(ascending=False, by='FWD_RETURN_5Y_FORECAST')
df.head()

,ETF_NAME,BENCHMARK_NAME,BENCHMARK_TICKER,FWD_RETURN_5Y_FORECAST
ETF_TICKER,,,,
PSCD,Invesco S&P SmallCap Consumer Discretionary ETF,S&P 600 Consumer Discretionary Sector GICS Lev...,S6COND Index,0.2189
SLYV,SPDR S&P 600 Small CapValue ETF,S&P Small Cap 600 Value Index,SMLV Index,0.1593
VIOV,Vanguard S&P Small-Cap 600 Value ETF,S&P Small Cap 600 Value Index,SMLV Index,0.1593
IJS,iShares S&P Small-Cap 600 Value ETF,S&P Small Cap 600 Value Index,SMLV Index,0.1593
PSCI,Invesco S&P SmallCap Industrials ETF,S&P 600 Industrials Sector GICS Level 1 Index,S6INDU Index,0.1317


In [5]:
benchmark_ticker = 'SPY'
print('BENCHMARK INDEX FORWARD 5-YEAR RETURN FORECAST')
print('{} ({}): {}%'.format(
    df.loc['SPY'].ETF_NAME,
    benchmark_ticker,
    results.loc[benchmark_ticker].FWD_RETURN_5Y_FORECAST * 100
    ))

BENCHMARK INDEX FORWARD 5-YEAR RETURN FORECAST
SPDR S&P 500 ETF Trust (SPY): -0.66%


In [6]:
# df[df.FWD_RETURN_5Y_FORECAST>=results.loc['SPY'].FWD_RETURN_5Y_FORECAST].sort_values(by='FWD_RETURN_5Y_FORECAST', ascending=False)
above_10_pct_exp_return = df[df.FWD_RETURN_5Y_FORECAST>=0.10].sort_values(by='FWD_RETURN_5Y_FORECAST', ascending=False)
above_10_pct_index_tickers = list(above_10_pct_exp_return.BENCHMARK_TICKER.drop_duplicates())

In [7]:
df[df['BENCHMARK_TICKER'].isin(above_10_pct_index_tickers)].drop_duplicates(subset=['BENCHMARK_TICKER']).sort_values(by='FWD_RETURN_5Y_FORECAST', ascending=False)

,ETF_NAME,BENCHMARK_NAME,BENCHMARK_TICKER,FWD_RETURN_5Y_FORECAST
ETF_TICKER,,,,
PSCD,Invesco S&P SmallCap Consumer Discretionary ETF,S&P 600 Consumer Discretionary Sector GICS Lev...,S6COND Index,0.2189
IJR,iShares Core S&P Small-Cap ETF,S&P Small Cap 600 Index,SML Index,0.1729
VIOG,Vanguard S&P Small-Cap 600 Growth ETF,S&P Small Cap 600 Growth Index,SMLG Index,0.1720
SLYV,SPDR S&P 600 Small CapValue ETF,S&P Small Cap 600 Value Index,SMLV Index,0.1593
QQQ,Invesco QQQ Trust Series 1,NASDAQ-100 Index,NDQ Index,0.1363
PSCI,Invesco S&P SmallCap Industrials ETF,S&P 600 Industrials Sector GICS Level 1 Index,S6INDU Index,0.1317
CXSE,WisdomTree Trust WisdomTree China ex-State-Own...,MSCI Zhong Hua Index,MXZH Index,0.1079
IEO,iShares U.S. Oil & Gas Exploration & Productio...,S&P Supercomposite Oil & Gas Exploration & Pro...,S15OILP Index,0.1073
MDYG,SPDR S&P 400 Mid CapGrowth ETF,S&P MidCap 400 Growth Index,MIDG Index,0.1038


In [8]:
df.loc['SPY']

ETF_NAME                                      SPDR S&P 500 ETF Trust
BENCHMARK_NAME            S&P 500 GICS Level 3 Industry Groups Index
BENCHMARK_TICKER                                         SPXL3 Index
FWD_RETURN_5Y_FORECAST                                       -0.0066
Name: SPY, dtype: object

In [9]:
benchmark_prices[df.BENCHMARK_TICKER]

,S6COND Index,SMLV Index,SMLV Index,SMLV Index,S6INDU Index,SML Index,SML Index,SML Index,SML Index,RUJ Index,...,MXUS0IT Index,MXUS0IT Index,S15UTIL Index,S15UTIL Index,S15UTIL Index,MXWO0TC Index,MXWOOTC Index,MXWOOTC Index,MXWOOTC Index,S5UTIL Index
date,,,,,,,,,,,,,,,,,,,,,
2002-09-30,150.90,139.44,139.44,139.44,180.17,187.86,187.86,187.86,187.86,535.054,...,39.91,39.91,104.51,104.51,104.51,33.09,32.90,32.90,32.90,94.42
2002-10-31,153.94,142.10,142.10,142.10,182.62,193.74,193.74,193.74,193.74,542.339,...,48.78,48.78,103.21,103.21,103.21,41.09,40.93,40.93,40.93,92.37
2002-11-29,162.72,149.58,149.58,149.58,191.68,203.65,203.65,203.65,203.65,584.616,...,57.18,57.18,104.62,104.62,104.62,45.55,45.42,45.42,45.42,94.25
2002-12-31,152.37,145.46,145.46,145.46,188.72,196.62,196.62,196.62,196.62,558.130,...,48.82,48.82,108.21,108.21,108.21,42.63,42.49,42.49,42.49,97.76
2003-01-31,146.34,139.34,139.34,139.34,174.41,189.75,189.75,189.75,189.75,541.776,...,48.19,48.19,104.41,104.41,104.41,41.65,41.49,41.49,41.49,94.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29,815.71,770.61,770.61,770.61,1420.35,1214.72,1214.72,1214.72,1214.72,2230.325,...,557.51,557.51,409.70,409.70,409.70,82.03,82.29,82.29,82.29,361.88
2022-05-31,789.01,786.89,786.89,786.89,1462.83,1235.61,1235.61,1235.61,1235.61,2268.092,...,547.34,547.34,426.23,426.23,426.23,83.02,83.33,83.33,83.33,375.72
2022-06-30,693.43,714.77,714.77,714.77,1349.43,1127.97,1127.97,1127.97,1127.97,2039.018,...,495.71,495.71,404.68,404.68,404.68,76.67,76.95,76.95,76.95,356.45


In [10]:
benchmark_lt_pe[df.BENCHMARK_TICKER]

,S6COND Index,SMLV Index,SMLV Index,SMLV Index,S6INDU Index,SML Index,SML Index,SML Index,SML Index,RUJ Index,...,MXUS0IT Index,MXUS0IT Index,S15UTIL Index,S15UTIL Index,S15UTIL Index,MXWO0TC Index,MXWOOTC Index,MXWOOTC Index,MXWOOTC Index,S5UTIL Index
date,,,,,,,,,,,,,,,,,,,,,
2002-09-30,22.02,16.01,16.01,16.01,18.32,21.29,21.29,21.29,21.29,14.34,...,22.92,22.92,9.99,9.99,9.99,8.61,8.52,8.52,8.52,9.46
2002-10-31,22.57,16.35,16.35,16.35,18.41,21.74,21.74,21.74,21.74,14.34,...,27.99,27.99,9.82,9.82,9.82,10.69,10.63,10.63,10.63,9.17
2002-11-29,23.70,17.38,17.38,17.38,19.03,22.85,22.85,22.85,22.85,15.44,...,32.13,32.13,9.91,9.91,9.91,13.78,13.80,13.80,13.80,9.15
2002-12-31,21.51,16.32,16.32,16.32,18.58,21.44,21.44,21.44,21.44,14.81,...,27.27,27.27,10.43,10.43,10.43,12.92,12.88,12.88,12.88,9.84
2003-01-31,19.79,15.21,15.21,15.21,16.69,20.10,20.10,20.10,20.10,14.12,...,26.07,26.07,9.70,9.70,9.70,12.80,12.76,12.76,12.76,9.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29,13.50,15.24,15.24,15.24,20.08,18.33,18.33,18.33,18.33,16.57,...,47.79,47.79,22.46,22.46,22.46,25.19,24.76,24.76,24.76,22.32
2022-05-31,13.05,15.42,15.42,15.42,20.37,18.70,18.70,18.70,18.70,16.94,...,47.16,47.16,23.37,23.37,23.37,25.23,24.85,24.85,24.85,23.41
2022-06-30,11.74,14.16,14.16,14.16,18.81,17.19,17.19,17.19,17.19,14.36,...,42.56,42.56,22.21,22.21,22.21,23.12,23.14,23.14,23.14,22.24


In [11]:
periodicity = 'MONTHLY'